# Download Sentinel2 bottom-of-atmosphere data

In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import IPython.display as disp
from time import sleep, strftime
from datetime import date, timedelta, datetime, timezone

from Py6S import *
import math
# from tgess.src.simulation_models.gee_atmcorr_S2.bin.atmospheric import Atmospheric

%matplotlib inline

tqdm.pandas()

In [3]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AY0e-g7L-uz9n_wMjldLclY_bFV1h0paxOULjNf-qbWuNng5k8sQ_xhTn_4

Successfully saved authorization token.


In [3]:
df = pd.read_csv("../data/processed/GBOV_RM07_correct_plot_coords.csv")
df = df.drop(columns=[df.columns[0]])
# df = df.iloc[240:250]
df

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/GBOV_RM07_correct_plot_coords.csv'

In [4]:
def retrieve_reflectance(x):
    """
    param x: Row of a pandas dataframe with columns: latitude, longitude, date
    """
    
    #Hardcode value keys to avoid unnecessary computation
    val_keys = ['AOT', 'B1', 'B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 
            'B9', 'CLOUD_MASK', 'CLOUD_SHADOW_MASK', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 
            'QA20', 'QA60', 'SCL', 'SHADOW_MASK', 'TCI_B', 'TCI_G', 'TCI_R', 'WVP']
    
    aoi = ee.Geometry.Point(x["longitude"], x["latitude"])

    try:
        img = ee.Image(ee.ImageCollection('COPERNICUS/S2_SR') 
                        .filterBounds(aoi)
                        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',30))
                        .filterDate(ee.Date(x["date_start"]), ee.Date(x["date_end"]))
                        .closest(x["date"]) 
                        .maskClouds(prob = 70)
                        .first()
    #                     .clip(aoi)
                      )
    except Exception as e:
#         print(e)
        for k in val_keys:
            x[k] = None
        x["S2A_date"] = None
        return x
    
#     print(img)
#     return
    values = img.reduceRegion(reducer=ee.Reducer.mean(), scale=30, geometry=aoi, maxPixels=1e9).getInfo()
    date = ee.Date(img.get('system:time_start')).format().getInfo()
    
    # Avoid spamming API, sleep 0.1 second every query
    sleep(0.1)

    for k in val_keys:
        x[k] = values[k]
    
    x["S2A_date"] = str(date)
    return x

def get_all_reflectances(df):
    df = df.progress_apply(retrieve_reflectance, axis=1)
    return df
    
# retrieve_reflectance(df.iloc[0])

In [60]:
out_df = get_all_reflectances(df)
timestamp = strftime("%Y%m%d-%H%M%S")
out_df.to_csv("GBOV_and_S2A_correct_coords_{}.csv".format(timestamp))

In [11]:
out_df.columns

Index(['GBOV_ID', 'Site', 'GROUND_DATA_PI', 'GROUND_DATA_PIs_Email',
       'GBOV_Email', 'Network', 'Elevation', 'IGBP_class', 'Lat_IS', 'Lon_IS',
       'TIME_IS', 'Version', 'PLOT_ID', 'LAIe_Miller_up', 'LAIe_Miller_up_err',
       'LAI_Miller_up', 'LAI_Miller_up_err', 'clumping_Miller_up',
       'clumping_Miller_up_err', 'LAIe_Warren_up', 'LAIe_Warren_up_err',
       'LAI_Warren_up', 'LAI_Warren_up_err', 'clumping_Warren_up',
       'clumping_Warren_up_err', 'LAIe_Miller_down', 'LAIe_Miller_down_err',
       'LAI_Miller_down', 'LAI_Miller_down_err', 'clumping_Miller_down',
       'clumping_Miller_down_err', 'LAIe_Warren_down', 'LAIe_Warren_down_err',
       'LAI_Warren_down', 'LAI_Warren_down_err', 'clumping_Warren_down',
       'clumping_Warren_down_err', 'up_flag', 'down_flag', 'LAI_Miller',
       'LAI_Warren', 'datetime', 'tiles', 'date', 'date_start', 'date_end',
       'first_tile', 'second_tile', 'AOT', 'B1', 'B11', 'B12', 'B2', 'B3',
       'B4', 'B5', 'B6', 'B7', 'B8', 'B

In [29]:
cols = ["Site", "PLOT_ID", "date", "Lat_IS", "Lon_IS", "S2A_date", "LAI_Warren"]

out_df[cols]

,Site,PLOT_ID,date,Lat_IS,Lon_IS,S2A_date,LAI_Warren
0,JonesEcologicalResearchCenter,JERC_003,2019-08-26,31.194839,-84.468777,2019-08-23T16:34:48,3.950
1,JonesEcologicalResearchCenter,JERC_004,2019-08-20,31.194839,-84.468777,2019-08-23T16:34:48,2.650
2,JonesEcologicalResearchCenter,JERC_005,2019-08-20,31.194839,-84.468777,2019-08-23T16:34:48,2.280
3,JonesEcologicalResearchCenter,JERC_006,2019-08-20,31.194839,-84.468777,2019-08-23T16:34:48,1.880
4,JonesEcologicalResearchCenter,JERC_007,2019-08-26,31.194839,-84.468777,2019-08-23T16:34:48,2.330
...,...,...,...,...,...,...,...
4013,Woodworth,WOOD_069,2019-08-20,47.128231,-99.241364,2019-08-19T17:40:54,0.529
4014,Woodworth,WOOD_069,2019-09-04,47.128231,-99.241364,2019-09-01T17:50:48,0.311
4015,Woodworth,WOOD_069,2019-09-18,47.128231,-99.241364,None,0.383
4016,Woodworth,WOOD_069,2019-10-04,47.128231,-99.241364,2019-10-06T17:50:47,0.650


In [61]:
out_df["S2A_date"].isna().sum()

3298

In [64]:
out_df.dropna().iloc[:10]

,GBOV_ID,Site,GROUND_DATA_PI,GROUND_DATA_PIs_Email,GBOV_Email,Network,Elevation,IGBP_class,Lat_IS,Lon_IS,...,QA10,QA20,QA60,SCL,SHADOW_MASK,TCI_B,TCI_G,TCI_R,WVP,S2A_date
567,GBOV_RM7_1581,LajasExperimentalStation,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,24,Grassland,18.02125,-67.0769,...,0.0,0.0,0.0,4.0,0.0,59.0,92.0,101.0,2079.0,2018-12-14T15:07:57
569,GBOV_RM7_1581,LajasExperimentalStation,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,24,Grassland,18.02125,-67.0769,...,0.0,0.0,0.0,4.0,0.0,25.0,56.0,72.0,1410.0,2019-01-08T15:08:01
572,GBOV_RM7_1581,LajasExperimentalStation,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,24,Grassland,18.02125,-67.0769,...,0.0,0.0,0.0,4.0,0.0,49.0,76.0,45.0,2687.0,2019-02-22T15:07:47
574,GBOV_RM7_1581,LajasExperimentalStation,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,24,Grassland,18.02125,-67.0769,...,0.0,0.0,0.0,4.0,0.0,44.0,85.0,73.0,1744.0,2019-03-19T15:08:03
575,GBOV_RM7_1581,LajasExperimentalStation,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,24,Grassland,18.02125,-67.0769,...,0.0,0.0,0.0,4.0,0.0,54.0,83.0,94.0,2802.0,2019-04-03T15:08:09
576,GBOV_RM7_1581,LajasExperimentalStation,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,24,Grassland,18.02125,-67.0769,...,0.0,0.0,0.0,4.0,0.0,80.0,112.0,135.0,2491.0,2019-04-18T15:07:51
578,GBOV_RM7_1581,LajasExperimentalStation,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,24,Grassland,18.02125,-67.0769,...,0.0,0.0,0.0,5.0,0.0,98.0,124.0,150.0,3972.0,2019-05-13T15:08:00
579,GBOV_RM7_1581,LajasExperimentalStation,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,24,Grassland,18.02125,-67.0769,...,0.0,0.0,0.0,4.0,0.0,76.0,95.0,109.0,3654.0,2019-06-02T15:07:57
580,GBOV_RM7_1581,LajasExperimentalStation,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,24,Grassland,18.02125,-67.0769,...,0.0,0.0,0.0,5.0,0.0,75.0,102.0,123.0,3273.0,2019-06-12T15:08:13
581,GBOV_RM7_1581,LajasExperimentalStation,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,24,Grassland,18.02125,-67.0769,...,0.0,0.0,0.0,5.0,0.0,87.0,116.0,144.0,4091.0,2019-06-27T15:07:56


In [5]:
def get_s2_cloud_collection(aoi, date_start, date_end):
    # From: https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
    # Import and filter S2 SR.
    s2_col = (ee.ImageCollection('COPERNICUS/S2')
        .filterBounds(aoi)
        .filterDate(date_start, date_end)
        )

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(date_start, date_end))

    # Join the filtered s2cloudless collection to the S2 collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_band(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability').rename('cloud_probability')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb]))


def reflectance_time_series(df, path="../data/processed/", filename="GBOV_toa_reflectance_time_series_per_plot.csv"):
    bands = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B10", "B11", "B12"]
    
    # For each unique plotID
    output = []
    
    unique_plots = df["plotID"].unique()
    
    missing_counts = {plotID:0 for plotID in unique_plots}
    
    for plotID in tqdm(unique_plots):
        sub_df = df[df["plotID"]==plotID]
                
        # Get first and last date, buffer 1 month around date
        date_start = datetime.strptime(sub_df["date"].min(), '%Y-%m-%d') - timedelta(days=30)
#         print(date_start)
        date_end = datetime.strptime(sub_df["date"].max(), '%Y-%m-%d') + timedelta(days=30)
#         print(date_end)
        # Get location
        aoi = ee.Geometry.Point(sub_df.iloc[0]["longitude"], sub_df.iloc[0]["latitude"])
        
        # Retrieve time series from GEE
        collection = get_s2_cloud_collection(aoi, date_start, date_end).map(add_cloud_band)
#         collection = ee.ImageCollection('COPERNICUS/S2')\
#                         .filterBounds(aoi)\
#                         .filterDate(ee.Date(date_start), ee.Date(date_end))\
#                         .maskClouds(prob = 70)\
        
        count = collection.size()
        
        if count.getInfo()>0:
#             collection = collection.maskClouds(prob=70)
            
            # Retrieve properties as time series (list)
            zenith_angle = collection.aggregate_array("MEAN_SOLAR_ZENITH_ANGLE").getInfo()
            azimuth_angle = collection.aggregate_array("MEAN_SOLAR_AZIMUTH_ANGLE").getInfo()
                       
            # Retrieve pixel values for region as time series (list of lists)
            values = collection.getRegion(geometry=aoi, scale=10)
            values = values.getInfo()
                        
            # Create dataframe
            values = pd.DataFrame(values)
            
            # First row is header
            values.columns = values.iloc[0]
            values = values[1:]
            
            # Create columns for properties time series
            values["zenith_angle"]=zenith_angle
            values["azimuth_angle"]=azimuth_angle
            values["plotID"] = plotID
            values["retrieval_datetime"]=pd.to_datetime(values["time"], unit='ms')
                        
            # Retrieve solar irradiance and observer angle per band as time series (list)
            for band in bands:
                # Solar irradiance
                values["solar_irradiance_{}".format(band)] = \
                collection.aggregate_array("SOLAR_IRRADIANCE_{}".format(band)).getInfo()
                
                # Incidence azimuth
                values["incidence_azimuth_{}".format(band)] = \
                collection.aggregate_array("MEAN_INCIDENCE_AZIMUTH_ANGLE_{}".format(band)).getInfo()
                
                # Incidence zenith
                values["incidence_zenith_{}".format(band)] = \
                collection.aggregate_array("MEAN_INCIDENCE_ZENITH_ANGLE_{}".format(band)).getInfo()
                
            values["mean_incidence_azimuth"] = values.loc[: , "incidence_azimuth_B1":"incidence_azimuth_B12"].mean(axis=1)
            values["mean_incidence_zenith"] = values.loc[: , "incidence_zenith_B1":"zenith_B12"].mean(axis=1)
            
            output.append(values)   
        
        else:
            print("Missing data for {} in range {} to {}".format(plotID, date_start, date_end))
            missing_counts[plotID]+=1

            
    output = pd.concat(output)
#     print(missing_counts)
    output.to_csv((path+filename))
    print("Saved results as {}{}".format(path, filename))
    return output


In [236]:
ts_df = reflectance_time_series(df)
path = "../data/processed/"
filename = "GBOV_toa_reflectance_time_series_per_plot.csv"
# ts_df = pd.read_csv("{}{}".format(path, filename))

  0%|          | 0/286 [00:00<?, ?it/s]

Missing data for ORNL_037 in range 2014-05-05 00:00:00 to 2014-11-06 00:00:00
Missing data for ORNL_057 in range 2014-05-05 00:00:00 to 2014-11-06 00:00:00
Missing data for OSBS_013 in range 2013-04-19 00:00:00 to 2013-06-18 00:00:00
Missing data for TALL_001 in range 2015-06-16 00:00:00 to 2015-08-15 00:00:00
Missing data for TALL_002 in range 2015-06-27 00:00:00 to 2015-08-26 00:00:00
Missing data for TALL_003 in range 2015-06-17 00:00:00 to 2015-08-16 00:00:00
Missing data for TALL_004 in range 2015-06-14 00:00:00 to 2015-08-13 00:00:00
Missing data for TALL_005 in range 2015-06-17 00:00:00 to 2015-08-16 00:00:00
Missing data for TALL_006 in range 2015-06-27 00:00:00 to 2015-08-26 00:00:00
Missing data for TALL_007 in range 2015-06-28 00:00:00 to 2015-08-27 00:00:00
Missing data for TALL_008 in range 2015-06-27 00:00:00 to 2015-08-26 00:00:00
Missing data for TALL_009 in range 2015-06-17 00:00:00 to 2015-08-16 00:00:00
Missing data for TALL_010 in range 2015-06-28 00:00:00 to 2015-0

In [237]:
ts_df.columns

Index(['id', 'longitude', 'latitude', 'time', 'B1', 'B2', 'B3', 'B4', 'B5',
       'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20',
       'QA60', 'cloud_probability', 'zenith_angle', 'azimuth_angle', 'plotID',
       'retrieval_datetime', 'solar_irradiance_B1', 'solar_irradiance_B2',
       'solar_irradiance_B3', 'solar_irradiance_B4', 'solar_irradiance_B5',
       'solar_irradiance_B6', 'solar_irradiance_B7', 'solar_irradiance_B8',
       'solar_irradiance_B8A', 'solar_irradiance_B9', 'solar_irradiance_B10',
       'solar_irradiance_B11', 'solar_irradiance_B12'],
      dtype='object', name=0)

In [186]:
ts_df[ts_df["plotID"]=="JERC_047"].drop_duplicates(subset=["retrieval_datetime"], keep="first")

# subset=['B1', 'B2', 'B3', 'B4', 'B5',
#        'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20',
#        'QA60', 'cloud_probability', 'plotID', "retrieval_datetime"
#        ]

,id,longitude,latitude,time,B1,B2,B3,B4,B5,B6,...,B12,QA10,QA20,QA60,cloud_probability,zenith_angle,azimuth_angle,plotID,retrieval_datetime,retrieval_date
1,20151003T163026_20151003T163029_T16RGV,-84.467194,31.198445,1443889829000,5531,5142,4824,5167,5053,5465,...,3402,0,0,1024,100,37.218904,158.438165,JERC_047,2015-10-03 16:30:29.000,2015-10-03
2,20151003T163026_20161206T130137_T16RGV,-84.467194,31.198445,1443889829461,5531,5142,4824,5167,5053,5465,...,3402,0,0,1024,100,37.218904,158.438165,JERC_047,2015-10-03 16:30:29.461,2015-10-03
3,20151020T162042_20151020T162304_T16RGV,-84.467194,31.198445,1445358184774,1192,853,728,470,839,1530,...,690,0,0,0,1,43.710994,159.419958,JERC_047,2015-10-20 16:23:04.774,2015-10-20
5,20151023T162332_20151023T163422_T16RGV,-84.467194,31.198445,1445618062000,1023,752,607,418,719,1436,...,553,0,0,0,0,44.023555,163.390351,JERC_047,2015-10-23 16:34:22.000,2015-10-23
6,20151023T162332_20161215T133519_T16RGV,-84.467194,31.198445,1445618062855,1023,752,607,418,719,1436,...,553,0,0,0,0,44.023555,163.390351,JERC_047,2015-10-23 16:34:22.855,2015-10-23
7,20151112T163022_20151112T163020_T16RGV,-84.467194,31.198445,1447345820000,1130,831,634,422,647,1231,...,511,0,0,0,1,50.116059,165.449988,JERC_047,2015-11-12 16:30:20.000,2015-11-12
8,20151112T163022_20161226T042522_T16RGV,-84.467194,31.198445,1447345820463,1130,831,634,422,647,1231,...,511,0,0,0,1,50.116059,165.449988,JERC_047,2015-11-12 16:30:20.463,2015-11-12
9,20151119T161542_20151119T161545_T16RGV,-84.467194,31.198445,1447949745000,7677,7523,7049,7361,7286,7437,...,2455,0,0,1024,100,52.475144,162.724284,JERC_047,2015-11-19 16:15:45.000,2015-11-19
10,20151119T161542_20170103T141208_T16RGV,-84.467194,31.198445,1447949745459,7677,7523,7049,7361,7286,7437,...,2455,0,0,1024,100,52.475144,162.724284,JERC_047,2015-11-19 16:15:45.459,2015-11-19


In [6]:
def nearest(df, date):
    # From: https://stackoverflow.com/questions/32237862/find-the-closest-date-to-a-given-date/32237949
    try:
        result = df.set_index("retrieval_datetime").sort_index().index.get_loc(date, method='nearest')
        return result

    except Exception as e:
        print(e)
        print(df)
        return None

In [7]:
def get_closest_cloud_free_data(x, ts_df):
    cols_to_drop = ['id', 'longitude', 'latitude', 'time', 'plotID']
    date = datetime.strptime(x["date"], "%Y-%m-%d")
    subset = ts_df[ts_df["plotID"]==x["plotID"]]
    subset = subset.drop(columns=cols_to_drop)
    cols = subset.columns
    nearest_idx = nearest(subset, date)

    if nearest_idx is None:
        # Concatenate None values
        x = pd.concat([x, pd.Series({col:None for col in cols})])
        x["abs_date_difference"] = None
        
    else:
        # Concatenate reflectance values from nearest date
        x = pd.concat([x, subset.iloc[nearest_idx]])
        x["abs_date_difference"] = abs(x["datetime"]-x["retrieval_datetime"]).days

    return x

In [8]:
def create_LAI_reflectance_dataset(df, ts_df, max_cloud_prb=70, max_date_diff=14, path="../data/processed/", filename="GBOV_LAI_toa_reflectances.csv"):
  
    # Drop duplicate rows, somehow does not work with float or datetime columns (rounding errors?)
    # But does work with date
    ts_df["retrieval_date"]=ts_df["retrieval_datetime"].apply(lambda x: x.date())
#     ts_df = ts_df.drop_duplicates(subset=['B1', 'B2', 'B3', 'B4', 'B5',
#        'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20',
#        'QA60', 'cloud_probability', 'plotID', 'retrieval_date'], ignore_index=True, keep="first")
    ts_df = ts_df.drop_duplicates(subset=["retrieval_datetime", "latitude", "longitude"], keep="first")

    df["datetime"] = pd.to_datetime(df["datetime"]).dt.tz_localize(None)
    #.dt.tz_convert(timezone.utc)
    ts_df["retrieval_datetime"] = pd.to_datetime(ts_df["retrieval_datetime"]).dt.tz_localize(None)
    #.dt.tz_convert(timezone.utc)
    #.dt.tz_localize(timezone.utc)
    
    
    ts_df = ts_df[ts_df["cloud_probability"]<max_cloud_prb]
    #     ts_df = ts_df[ts_df["MSK_CLDPRB"]<max_cloud_prb]
    
    df = df.progress_apply(get_closest_cloud_free_data, args=(ts_df,), axis=1)
    
    n_unfiltered = len(df)
    df.to_csv(filename[:-4]+"_unfiltered.csv")
    
    df = df[~df["retrieval_datetime"].isna()]
    display(df)
    n_dropna = len(df)
    df.to_csv(filename[:-4]+"_only_valid_measurements.csv")
    print("Lost {} samples from dropping NaN measurements.".format(n_unfiltered-n_dropna))
    
    df = df[df["abs_date_difference"]<=max_date_diff]
    df.to_csv(filename[:-4]+"_filtered.csv")
    n_date_diff = len(df)
    print("Lost {} samples from filtering on date difference.".format(n_dropna-n_date_diff))
    
    return df

In [241]:
out_df = create_LAI_reflectance_dataset(df, ts_df)

<ipython-input-240-6cc82031e857>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts_df["retrieval_datetime"] = pd.to_datetime(ts_df["retrieval_datetime"]).dt.tz_localize(None)


  0%|          | 0/3970 [00:00<?, ?it/s]

datetime.datetime(2014, 6, 4, 0, 0)
Empty DataFrame
Columns: [B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60, cloud_probability, zenith_angle, azimuth_angle, retrieval_datetime, solar_irradiance_B1, solar_irradiance_B2, solar_irradiance_B3, solar_irradiance_B4, solar_irradiance_B5, solar_irradiance_B6, solar_irradiance_B7, solar_irradiance_B8, solar_irradiance_B8A, solar_irradiance_B9, solar_irradiance_B10, solar_irradiance_B11, solar_irradiance_B12, retrieval_date]
Index: []

[0 rows x 34 columns]
datetime.datetime(2014, 6, 18, 0, 0)
Empty DataFrame
Columns: [B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60, cloud_probability, zenith_angle, azimuth_angle, retrieval_datetime, solar_irradiance_B1, solar_irradiance_B2, solar_irradiance_B3, solar_irradiance_B4, solar_irradiance_B5, solar_irradiance_B6, solar_irradiance_B7, solar_irradiance_B8, solar_irradiance_B8A, solar_irradiance_B9, solar_irradiance_B10, solar_irradiance_B11, solar_irra

datetime.datetime(2013, 5, 19, 0, 0)
Empty DataFrame
Columns: [B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60, cloud_probability, zenith_angle, azimuth_angle, retrieval_datetime, solar_irradiance_B1, solar_irradiance_B2, solar_irradiance_B3, solar_irradiance_B4, solar_irradiance_B5, solar_irradiance_B6, solar_irradiance_B7, solar_irradiance_B8, solar_irradiance_B8A, solar_irradiance_B9, solar_irradiance_B10, solar_irradiance_B11, solar_irradiance_B12, retrieval_date]
Index: []

[0 rows x 34 columns]
datetime.datetime(2016, 7, 27, 0, 0)
Empty DataFrame
Columns: [B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60, cloud_probability, zenith_angle, azimuth_angle, retrieval_datetime, solar_irradiance_B1, solar_irradiance_B2, solar_irradiance_B3, solar_irradiance_B4, solar_irradiance_B5, solar_irradiance_B6, solar_irradiance_B7, solar_irradiance_B8, solar_irradiance_B8A, solar_irradiance_B9, solar_irradiance_B10, solar_irradiance_B11, solar_irr

datetime.datetime(2014, 5, 20, 0, 0)
Empty DataFrame
Columns: [B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60, cloud_probability, zenith_angle, azimuth_angle, retrieval_datetime, solar_irradiance_B1, solar_irradiance_B2, solar_irradiance_B3, solar_irradiance_B4, solar_irradiance_B5, solar_irradiance_B6, solar_irradiance_B7, solar_irradiance_B8, solar_irradiance_B8A, solar_irradiance_B9, solar_irradiance_B10, solar_irradiance_B11, solar_irradiance_B12, retrieval_date]
Index: []

[0 rows x 34 columns]
datetime.datetime(2014, 6, 3, 0, 0)
Empty DataFrame
Columns: [B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60, cloud_probability, zenith_angle, azimuth_angle, retrieval_datetime, solar_irradiance_B1, solar_irradiance_B2, solar_irradiance_B3, solar_irradiance_B4, solar_irradiance_B5, solar_irradiance_B6, solar_irradiance_B7, solar_irradiance_B8, solar_irradiance_B8A, solar_irradiance_B9, solar_irradiance_B10, solar_irradiance_B11, solar_irra

datetime.datetime(2015, 7, 1, 0, 0)
Empty DataFrame
Columns: [B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60, cloud_probability, zenith_angle, azimuth_angle, retrieval_datetime, solar_irradiance_B1, solar_irradiance_B2, solar_irradiance_B3, solar_irradiance_B4, solar_irradiance_B5, solar_irradiance_B6, solar_irradiance_B7, solar_irradiance_B8, solar_irradiance_B8A, solar_irradiance_B9, solar_irradiance_B10, solar_irradiance_B11, solar_irradiance_B12, retrieval_date]
Index: []

[0 rows x 34 columns]
datetime.datetime(2015, 7, 15, 0, 0)
Empty DataFrame
Columns: [B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60, cloud_probability, zenith_angle, azimuth_angle, retrieval_datetime, solar_irradiance_B1, solar_irradiance_B2, solar_irradiance_B3, solar_irradiance_B4, solar_irradiance_B5, solar_irradiance_B6, solar_irradiance_B7, solar_irradiance_B8, solar_irradiance_B8A, solar_irradiance_B9, solar_irradiance_B10, solar_irradiance_B11, solar_irra

,GBOV_ID,Site,GROUND_DATA_PI,GROUND_DATA_PIs_Email,GBOV_Email,Network,Elevation,IGBP_class,Lat_IS,Lon_IS,...,solar_irradiance_B6,solar_irradiance_B7,solar_irradiance_B8,solar_irradiance_B8A,solar_irradiance_B9,solar_irradiance_B10,solar_irradiance_B11,solar_irradiance_B12,retrieval_date,abs_date_difference
0,GBOV_RM7_1566,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,44,Croplands,31.194839,-84.468777,...,1287.61,1162.08,1041.63,955.32,812.92,367.15,245.59,85.25,2019-08-23,3.0
1,GBOV_RM7_1563,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,44,Croplands,31.194839,-84.468777,...,1287.61,1162.08,1041.63,955.32,812.92,367.15,245.59,85.25,2019-08-23,3.0
2,GBOV_RM7_1542,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,44,Croplands,31.194839,-84.468777,...,1291.13,1175.57,1041.28,953.93,817.58,365.41,247.08,87.75,2019-08-15,4.0
3,GBOV_RM7_1521,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,44,Croplands,31.194839,-84.468777,...,1287.61,1162.08,1041.63,955.32,812.92,367.15,245.59,85.25,2019-08-23,2.0
4,GBOV_RM7_1572,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,44,Croplands,31.194839,-84.468777,...,1287.61,1162.08,1041.63,955.32,812.92,367.15,245.59,85.25,2019-08-23,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3965,GBOV_RM7_808,Woodworth,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,579,Croplands,47.128231,-99.241364,...,1291.13,1175.57,1041.28,953.93,817.58,365.41,247.08,87.75,2019-08-19,0.0
3966,GBOV_RM7_808,Woodworth,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,579,Croplands,47.128231,-99.241364,...,1287.61,1162.08,1041.63,955.32,812.92,367.15,245.59,85.25,2019-09-06,2.0
3967,GBOV_RM7_808,Woodworth,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,579,Croplands,47.128231,-99.241364,...,1291.13,1175.57,1041.28,953.93,817.58,365.41,247.08,87.75,2019-09-18,0.0
3968,GBOV_RM7_808,Woodworth,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,579,Croplands,47.128231,-99.241364,...,1287.61,1162.08,1041.63,955.32,812.92,367.15,245.59,85.25,2019-10-06,2.0


Lost 96 samples from dropping NaN measurements.
Lost 864 samples from filtering on date difference.


In [222]:
path="../data/processed"
filename="GBOV_LAI_toa_reflectances.csv"
out_df = pd.read_csv(filename[:-4]+"_filtered.csv").drop(columns=out_df.columns[0])
out_df

,Unnamed: 0,Site,GROUND_DATA_PI,GROUND_DATA_PIs_Email,GBOV_Email,Network,Elevation,IGBP_class,Lat_IS,Lon_IS,...,B12,QA10,QA20,QA60,cloud_probability,zenith_angle,azimuth_angle,retrieval_datetime,retrieval_date,abs_date_difference
0,0,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,44,Croplands,31.194839,-84.468777,...,448.0,0.0,0.0,0.0,0.0,24.824025,139.116807,2019-08-23 16:34:48.893,2019-08-23,3.0
1,1,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,44,Croplands,31.194839,-84.468777,...,717.0,0.0,0.0,0.0,2.0,24.824025,139.116807,2019-08-23 16:34:48.893,2019-08-23,3.0
2,2,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,44,Croplands,31.194839,-84.468777,...,703.0,0.0,0.0,0.0,5.0,24.578535,129.846406,2019-08-15 16:24:57.531,2019-08-15,4.0
3,3,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,44,Croplands,31.194839,-84.468777,...,716.0,0.0,0.0,0.0,36.0,24.824025,139.116807,2019-08-23 16:34:48.893,2019-08-23,2.0
4,4,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,44,Croplands,31.194839,-84.468777,...,1595.0,0.0,0.0,0.0,67.0,24.824025,139.116807,2019-08-23 16:34:48.893,2019-08-23,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,3965,Woodworth,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,579,Croplands,47.128231,-99.241364,...,1106.0,0.0,0.0,0.0,1.0,36.952902,154.686462,2019-08-19 17:40:54.546,2019-08-19,0.0
3006,3966,Woodworth,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,579,Croplands,47.128231,-99.241364,...,1370.0,0.0,0.0,0.0,1.0,42.176381,162.841375,2019-09-06 17:50:44.641,2019-09-06,2.0
3007,3967,Woodworth,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,579,Croplands,47.128231,-99.241364,...,1903.0,0.0,0.0,0.0,63.0,46.941531,162.175233,2019-09-18 17:40:47.867,2019-09-18,0.0
3008,3968,Woodworth,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,579,Croplands,47.128231,-99.241364,...,1230.0,0.0,0.0,0.0,4.0,53.152337,168.873554,2019-10-06 17:50:47.641,2019-10-06,2.0


In [9]:
def get_atmosphere_features(x):
    date = ee.Date(str(x["retrieval_date"]))
    aoi = ee.Geometry.Point(x["longitude"], x["latitude"])

    x["atmosphere_water"] = Atmospheric.water(aoi, date).getInfo()
    x["atmosphere_ozone"] = Atmospheric.ozone(aoi, date).getInfo()
    x["atmosphere_aerosol"] = Atmospheric.aerosol(aoi, date).getInfo()
    
    return x


def get_all_atmosphere_features(df, path="../data/processed/", filename="GBOV_LAI_toa_reflectances.csv"):   
    df = df.progress_apply(get_atmosphere_features, axis=1)
    
    df.to_csv(path+filename[:-4]+"_atmospheric_features.csv")
    
    return df

In [249]:
out_df = get_all_atmosphere_features(out_df)

  0%|          | 0/3010 [00:00<?, ?it/s]

In [10]:
def spectralResponseFunction(bandname):
    """
    Extract spectral response function for given band name
    """
    bandSelect = {
        'B1':PredefinedWavelengths.S2A_MSI_01,
        'B2':PredefinedWavelengths.S2A_MSI_02,
        'B3':PredefinedWavelengths.S2A_MSI_03,
        'B4':PredefinedWavelengths.S2A_MSI_04,
        'B5':PredefinedWavelengths.S2A_MSI_05,
        'B6':PredefinedWavelengths.S2A_MSI_06,
        'B7':PredefinedWavelengths.S2A_MSI_07,
        'B8':PredefinedWavelengths.S2A_MSI_08,
        'B8A':PredefinedWavelengths.S2A_MSI_8A,
        'B9':PredefinedWavelengths.S2A_MSI_09,
        'B10':PredefinedWavelengths.S2A_MSI_10,
        'B11':PredefinedWavelengths.S2A_MSI_11,
        'B12':PredefinedWavelengths.S2A_MSI_12,
        }
    return Wavelength(bandSelect[bandname])

def toa_to_rad(bandname, solar_irradiance, toa_reflectance, date, s):
    """
    Converts top of atmosphere reflectance to at-sensor radiance
    """
    
    # solar exoatmospheric spectral irradiance
    solar_angle_correction = math.cos(math.radians(s.geometry.solar_z))
    
    # Earth-Sun distance (from day of year)
    doy = date.timetuple().tm_yday # day of year
    d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
   
    # conversion factor
    multiplier = solar_irradiance*solar_angle_correction/(math.pi*d**2)

    # at-sensor radiance
    rad = toa_reflectance*multiplier
    
    return rad

def surface_reflectance(bandname, solar_irradiance, toa_reflectance, date, s):
    """
    Calculate surface reflectance from at-sensor radiance given waveband name
    """
    
    # run 6S for this waveband
    s.wavelength = spectralResponseFunction(bandname)
    s.run()
    
    # extract 6S outputs
    Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
    Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
    Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
    absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
    scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
    tau2 = absorb*scatter                                #total transmissivity
    
    # radiance to surface reflectance
    rad = toa_to_rad(bandname, solar_irradiance, toa_reflectance, date, s)
    ref = ((rad-Lp)*math.pi)/(tau2*(Edir+Edif))
    
    return ref


def apply_atmospheric_correction(x, bands):
    s = SixS()

    # Atmospheric constituents
    s.atmos_profile = AtmosProfile.UserWaterAndOzone(x["atmosphere_water"],x["atmosphere_ozone"])
    s.aero_profile = AeroProfile.Continental
    s.aot550 = x["atmosphere_aerosol"]

    # Earth-Sun-satellite geometry
    s.geometry = Geometry.User()
    s.geometry.view_z = 0               # always NADIR (I think..)
    s.geometry.solar_z = x["zenith_angle"]        # solar zenith angle
    s.geometry.month = x["retrieval_date"].month # month used for Earth-Sun distance
    s.geometry.day = x["retrieval_date"].day     # day used for Earth-Sun distance
    s.altitudes.set_sensor_satellite_level()
    s.altitudes.set_target_custom_altitude(x["Elevation"]) # elevation in km
    
    for band in bands:
        x[band] = surface_reflectance(band, x["solar_irradiance_{}".format(band)], x[band], x["retrieval_date"], s)
        
    del s
    return x

def create_atm_corrected_data(df, path="../data/processed/", filename="GBOV_LAI_toa_reflectances_6S_corrected.csv"):
    bands = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B10", "B11", "B12"]
        
    df["retrieval_date"] = pd.to_datetime(df["retrieval_date"])
    # Correct reflectance values (Sentinel2 values are upscaled by factor 10000)
    for band in bands:
        df[band] = df[band].apply(lambda x: x/10000)

    # Convert elevation from meters to kilometers
    df["Elevation"] = df["Elevation"].apply(lambda x: x/1000)
    
    df = df.progress_apply(apply_atmospheric_correction, args=(bands,), axis=1)
    
    print("Saving results to file: {}{}.".format(path, filename))
    df.to_csv(path+filename)
    
    return df

In [323]:
test_df = pd.read_csv("../data/processed/GBOV_LAI_toa_reflectances_atmospheric_features.csv")
test_df = create_atm_corrected_data(test_df)


  0%|          | 0/3010 [00:00<?, ?it/s]

Saving results to file: ../data/processed/GBOV_LAI_toa_reflectances_6S_corrected.csv.


In [329]:
test_df[["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B10", "B11", "B12"]]

,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B10,B11,B12
0,0.001565,0.004457,0.018777,0.005868,0.053885,0.215227,0.272878,0.279022,0.299037,0.415712,3.256321,0.147986,0.056031
1,0.003642,0.012041,0.057637,0.011670,0.110553,0.481353,0.549242,0.564990,0.578263,0.881432,28.400948,0.226654,0.091035
2,0.034019,0.033172,0.059369,0.038640,0.098353,0.283815,0.350640,0.347076,0.370894,0.406178,23.288713,0.201120,0.095363
3,0.104134,0.090857,0.099572,0.082325,0.117139,0.237315,0.273843,0.273533,0.293866,0.453527,28.400948,0.154542,0.090905
4,0.095986,0.161643,0.186517,0.182505,0.222106,0.382559,0.417519,0.429370,0.450486,0.722211,28.400948,0.288875,0.205285
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,0.020699,0.030207,0.060030,0.049434,0.110304,0.265208,0.322378,0.328252,0.353491,0.396886,0.809297,0.253896,0.131672
3006,0.026859,0.039025,0.063793,0.070252,0.122732,0.229230,0.252084,0.276390,0.301064,0.484247,0.902948,0.275414,0.154440
3007,0.111281,0.103496,0.117937,0.113772,0.174586,0.296374,0.343483,0.377032,0.383349,0.579265,4.722932,0.390261,0.227928
3008,0.039922,0.045057,0.068648,0.061333,0.119995,0.245422,0.277254,0.300696,0.316664,0.450818,0.584663,0.264280,0.143221


In [332]:
test_df["B10"].mean()

26.285500070328585

In [324]:
test_df

,Unnamed: 0,GBOV_ID,Site,GROUND_DATA_PI,GROUND_DATA_PIs_Email,GBOV_Email,Network,Elevation,IGBP_class,Lat_IS,...,solar_irradiance_B8A,solar_irradiance_B9,solar_irradiance_B10,solar_irradiance_B11,solar_irradiance_B12,retrieval_date,abs_date_difference,atmosphere_water,atmosphere_ozone,atmosphere_aerosol
0,0,GBOV_RM7_1566,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,0.044,Croplands,31.194839,...,955.32,812.92,367.15,245.59,85.25,2019-08-23,3.0,4.76,0.292000,0.356000
1,1,GBOV_RM7_1563,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,0.044,Croplands,31.194839,...,955.32,812.92,367.15,245.59,85.25,2019-08-23,3.0,4.76,0.292000,0.356000
2,2,GBOV_RM7_1542,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,0.044,Croplands,31.194839,...,953.93,817.58,365.41,247.08,87.75,2019-08-15,4.0,4.95,0.304052,0.356000
3,3,GBOV_RM7_1521,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,0.044,Croplands,31.194839,...,955.32,812.92,367.15,245.59,85.25,2019-08-23,2.0,4.76,0.292000,0.356000
4,4,GBOV_RM7_1572,JonesEcologicalResearchCenter,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,0.044,Croplands,31.194839,...,955.32,812.92,367.15,245.59,85.25,2019-08-23,3.0,4.76,0.292000,0.356000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,3965,GBOV_RM7_808,Woodworth,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,0.579,Croplands,47.128231,...,953.93,817.58,365.41,247.08,87.75,2019-08-19,0.0,1.88,0.326000,0.204667
3006,3966,GBOV_RM7_808,Woodworth,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,0.579,Croplands,47.128231,...,955.32,812.92,367.15,245.59,85.25,2019-09-06,2.0,1.86,0.262000,0.066333
3007,3967,GBOV_RM7_808,Woodworth,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,0.579,Croplands,47.128231,...,953.93,817.58,365.41,247.08,87.75,2019-09-18,0.0,2.67,0.268000,0.066333
3008,3968,GBOV_RM7_808,Woodworth,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,0.579,Croplands,47.128231,...,955.32,812.92,367.15,245.59,85.25,2019-10-06,2.0,1.17,0.316000,0.077333


In [325]:
test_df.columns

Index(['Unnamed: 0', 'GBOV_ID', 'Site', 'GROUND_DATA_PI',
       'GROUND_DATA_PIs_Email', 'GBOV_Email', 'Network', 'Elevation',
       'IGBP_class', 'Lat_IS', 'Lon_IS', 'TIME_IS', 'Version', 'PLOT_ID',
       'LAIe_Miller_up', 'LAIe_Miller_up_err', 'LAI_Miller_up',
       'LAI_Miller_up_err', 'clumping_Miller_up', 'clumping_Miller_up_err',
       'LAIe_Warren_up', 'LAIe_Warren_up_err', 'LAI_Warren_up',
       'LAI_Warren_up_err', 'clumping_Warren_up', 'clumping_Warren_up_err',
       'LAIe_Miller_down', 'LAIe_Miller_down_err', 'LAI_Miller_down',
       'LAI_Miller_down_err', 'clumping_Miller_down',
       'clumping_Miller_down_err', 'LAIe_Warren_down', 'LAIe_Warren_down_err',
       'LAI_Warren_down', 'LAI_Warren_down_err', 'clumping_Warren_down',
       'clumping_Warren_down_err', 'up_flag', 'down_flag', 'LAI_Miller',
       'LAI_Warren', 'datetime', 'tiles', 'date', 'date_start', 'date_end',
       'first_tile', 'second_tile', 'plotID', 'longitude', 'latitude', 'B1',
       'B2', 'B3

In [328]:
cols = ['Site', 'date', 'plotID', 'retrieval_date', 'LAI_Miller', 'LAI_Warren', 'B1',
       'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11',
       'B12', 'QA10', 'QA20', 'QA60', 'cloud_probability']

test_df[cols].to_csv("../data/processed/GBOV_LAI_final_dataset.csv")

In [343]:
band_cols = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']

In [352]:
test_df[band_cols] = test_df[band_cols].clip(lower=0)

In [354]:
test_df.columns

Index(['Unnamed: 0', 'GBOV_ID', 'Site', 'GROUND_DATA_PI',
       'GROUND_DATA_PIs_Email', 'GBOV_Email', 'Network', 'Elevation',
       'IGBP_class', 'Lat_IS', 'Lon_IS', 'TIME_IS', 'Version', 'PLOT_ID',
       'LAIe_Miller_up', 'LAIe_Miller_up_err', 'LAI_Miller_up',
       'LAI_Miller_up_err', 'clumping_Miller_up', 'clumping_Miller_up_err',
       'LAIe_Warren_up', 'LAIe_Warren_up_err', 'LAI_Warren_up',
       'LAI_Warren_up_err', 'clumping_Warren_up', 'clumping_Warren_up_err',
       'LAIe_Miller_down', 'LAIe_Miller_down_err', 'LAI_Miller_down',
       'LAI_Miller_down_err', 'clumping_Miller_down',
       'clumping_Miller_down_err', 'LAIe_Warren_down', 'LAIe_Warren_down_err',
       'LAI_Warren_down', 'LAI_Warren_down_err', 'clumping_Warren_down',
       'clumping_Warren_down_err', 'up_flag', 'down_flag', 'LAI_Miller',
       'LAI_Warren', 'datetime', 'tiles', 'date', 'date_start', 'date_end',
       'first_tile', 'second_tile', 'plotID', 'longitude', 'latitude', 'B1',
       'B2', 'B3

In [361]:
cols = ['Site', 'date', 'plotID', 'retrieval_date', 'LAI_Miller', 'LAI_Warren', 'B1',
       'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11',
       'B12']

test_df[cols].to_csv("../data/processed/GBOV_LAI_final_dataset_corrected.csv")

## Add incidence angles

In [11]:
df = pd.read_csv("../data/processed/GBOV_LAI_toa_reflectances_6S_corrected.csv")

In [20]:
df.columns

Index(['GBOV_ID', 'Site', 'GROUND_DATA_PI', 'GROUND_DATA_PIs_Email',
       'GBOV_Email', 'Network', 'Elevation', 'IGBP_class', 'Lat_IS', 'Lon_IS',
       'TIME_IS', 'Version', 'PLOT_ID', 'LAIe_Miller_up', 'LAIe_Miller_up_err',
       'LAI_Miller_up', 'LAI_Miller_up_err', 'clumping_Miller_up',
       'clumping_Miller_up_err', 'LAIe_Warren_up', 'LAIe_Warren_up_err',
       'LAI_Warren_up', 'LAI_Warren_up_err', 'clumping_Warren_up',
       'clumping_Warren_up_err', 'LAIe_Miller_down', 'LAIe_Miller_down_err',
       'LAI_Miller_down', 'LAI_Miller_down_err', 'clumping_Miller_down',
       'clumping_Miller_down_err', 'LAIe_Warren_down', 'LAIe_Warren_down_err',
       'LAI_Warren_down', 'LAI_Warren_down_err', 'clumping_Warren_down',
       'clumping_Warren_down_err', 'up_flag', 'down_flag', 'LAI_Miller',
       'LAI_Warren', 'datetime', 'tiles', 'date', 'date_start', 'date_end',
       'first_tile', 'second_tile', 'plotID', 'longitude', 'latitude', 'B1',
       'B2', 'B3', 'B4', 'B5', 'B6', 

In [21]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df[["latitude", "longitude", "retrieval_date"]]

,latitude,longitude,retrieval_date
0,31.196146,-84.473242,2019-08-23
1,31.189774,-84.465861,2019-08-23
2,31.195968,-84.451228,2019-08-15
3,31.187671,-84.455850,2019-08-23
4,31.198632,-84.476315,2019-08-23
...,...,...,...
3005,47.128829,-99.243545,2019-08-19
3006,47.128829,-99.243545,2019-09-06
3007,47.128829,-99.243545,2019-09-18
3008,47.128829,-99.243545,2019-10-06


In [ ]:
def get_incidence_angles(x):
    bands = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B10", "B11", "B12"]

    aoi = ee.Geometry.Point(x["longitude"], x["latitude"])
    
    start_date = datetime.strptime(x["retrieval_date"], '%Y-%m-%d')
    end_date = datetime.strptime(x["retrieval_date"], '%Y-%m-%d') + timedelta(days=1)
      
    img = ee.ImageCollection('COPERNICUS/S2')\
                    .filterBounds(aoi)\
                    .filterDate(ee.Date(start_date), ee.Date(end_date))\
                    .first()

    # Retrieve observer angle per band as time series (list)
    for band in bands:

        # Incidence azimuth
        x["incidence_azimuth_{}".format(band)] = \
        img.get("MEAN_INCIDENCE_AZIMUTH_ANGLE_{}".format(band)).getInfo()

        # Incidence zenith
        x["incidence_zenith_{}".format(band)] = \
        img.get("MEAN_INCIDENCE_ZENITH_ANGLE_{}".format(band)).getInfo()

    x["mean_incidence_azimuth"] = x[["incidence_azimuth_{}".format(band) for band in bands]].mean()
    x["mean_incidence_zenith"] = x[["incidence_zenith_{}".format(band) for band in bands]].mean()

    return x
    
df = df.progress_apply(get_incidence_angles, axis=1)
df.to_csv("../data/processed/GBOV_LAI_angles.csv")

  0%|          | 0/3010 [00:00<?, ?it/s]

### Merge angle df with nlcd df

In [2]:
df = pd.read_csv("../data/processed/GBOV_LAI_angles.csv")

In [3]:
nlcd_df = pd.read_csv("../data/processed/GBOV_LAI_final_dataset_correct_NLCD.csv")

In [22]:
nlcd_df = nlcd_df.set_index(nlcd_df.columns[0])
nlcd_df.index.rename("index", inplace=True)

In [27]:
nlcd_df.join(df[["mean_incidence_azimuth", "mean_incidence_zenith", "zenith_angle",
 "azimuth_angle"]], how="left").to_csv("../data/processed/GBOV_LAI_final.csv")